In [1]:
%pylab inline
%config IPCompleter.greedy=True
import os  #  file system operations
import pandas as pd # pandas... the best time series library out there
import datetime as dt # date and time functions
import time
import sqlite3

Populating the interactive namespace from numpy and matplotlib


Список бумаг с которыми будем работать

In [2]:
SP_SYMBOLS =['AAPL','MSFT','AMZN','FB','JPM','JNJ','GOOG','GOOGL','XOM','BAC','WFC','INTC','T','V','CSCO','CVX','UNH','PFE','HD','PG','VZ','C','ABBV','BA','KO','CMCSA','MA','PM','PEP','ORCL','DIS','MRK','NVDA','MMM','AMGN','IBM','NFLX','WMT','MO','MCD','GE','HON','MDT','ABT','TXN','BMY','ADBE','UNP','GILD','BKNG','AVGO','ACN','UTX','GS','SLB','CAT','PYPL','QCOM','CRM','NKE','TMO','USB','SBUX','LMT','COST','MS','PNC','LLY','UPS','TWX','NEE','CELG','LOW','BLK','CVS','AXP','MU','CHTR','SCHW','MDLZ','CB','COP','AMAT','DHR','AMT','CL','GD','FDX','RTN','WBA','NOC','BIIB','BDX','ANTM','AET','EOG','BK','ATVI','CME','AGN','MON','SYK','DUK','ITW','ADP','TJX','DE','CSX','SPGI','AIG','MET','CTSH','OXY','ISRG','SPG','GM','COF','PRU','D','PX','EMR','CCI','ICE','VRTX','SO','ESRX','MMC','MAR','INTU','F','EBAY','ZTS','NSC','VLO','CI','PSX','HAL','KHC','KMB','STT','FOXA','HPQ','STZ','EA','HUM','BSX','TGT','TRV','DAL','APD','TEL','ILMN','AON','LRCX','JCI','EXC','ETN','ECL','AFL','LYB','WM','PGR','ADI','ALL','STI','SHW','BAX','MPC','PLD','FIS','MCK','LUV','EL','APC','AEP','EQIX','KMI','FISV','WDC','HPE','DXC','ADSK','PPG','ROST','EW','GIS','PSA','HCA','SYY','MTB','PXD','ROP','APH','TROW','DFS','MCO','YUM','RHT','SRE','CCL','SYF','ALXN','WY','REGN','GLW','CMI','ADM','FCX','APTV','MNST','AAL','VFC','FITB','PH','PEG','ROK','FTV','SWK','AMP','OKE','ZBH','PCAR','ED','KEY','DG','NTRS','WMB','PCG','MCHP','RF','A','CAH','MYL','IP','CFG','TSN','IR','DLTR','CXO','AVB','XEL','COL','WLTW','EQR','DLR','RCL','KR','NUE','ORLY','PAYX','SWKS','SBAC','NEM','EIX','WELL','CERN','HIG','ALGN','BXP','WEC','XLNX','CBS','KLAC','PPL','UAL','GPN','MGM','BBY','AME','DTE','VTR','IDXX','AZO','INFO','ES','MSI','LH','NTAP','CNC','HBAN','CMA','STX','K','OMC','DVN','WRK','CTL','LNC','SYMC','WAT','CLX','ABC','PFG','INCY','HLT','FAST','LEN','VRSK','FOX','EMN','VMC','MTD','RSG','ETFC','CAG','TXT','MHK','URI','DOV','ESS','IQV','DHI','CTAS','TSS','SJM','TPR','XL','WYNN','TAP','EFX','NBL','FE','BLL','O','DGX','GWW','AWK','TDG','HSY','ANSS','NOV','L','XYL','IVZ','RMD','BF.B','HST','NWL','ETR','CTXS','CBOE','GPC','MAS','EXPE','APA','XRAY','MLM','FTI','SNPS','MKC','AEE','CHD','BEN','EQT','CHRW','HES','AJG','AKAM','RJF','MRO','COG','CMS','ARE','ULTA','WHR','LKQ','HII','DRI','VAR','VNO','KMX','KSU','EXPD','PNR','CNP','VIAB','PVH','UNM','PRGO','COO','IT','ALB','CA','KSS','PKG','ZION','CINF','NCLH','ADS','CDNS','NLSN','IFF','AMG','FMC','UHS','DVA','HOLX','EXR','HSIC','VRSN','RE','JBHT','QRVO','ARNC','LB','TIF','AVY','HAS','BWA','MAA','SLG','JNPR','NDAQ','IPGP','AOS','MOS','FFIV','WU','FBHS','AMD','UDR','CF','IPG','DISH','NRG','M','DRE','COTY','HRL','LNT','IRM','SNA','XEC','REG','AAP','FRT','CPB','ALK','PNW','PKI','TSCO','PHM','CMG','ALLE','SEE','FLR','HOG','NI','RHI','HBI','GPS','JEC','GT','DISCK','GRMN','FLIR','AES','PBCT','HP','XRX','CSRA','AYI','MAC','LEG','AIV','NWSA','RL','KIM','JWN','FLS','HRB','FL','SRCL','PWR','BHF','EVHC','AIZ','MAT','TRIP','NFX','DISCA','NAVI','RRC','UAA','SIG','CHK','UAA','PDCO','NWS']

Период для создания обучающей выборки (должен быть как минимум на год меньше последнего года в базе)

In [3]:
year = 2020

Имя базы

In [4]:
base = 'invest2.db'

Функция получения исторических цен за год по бумаге

In [5]:
def get_price_history(base,symbol,year):
    conn = sqlite3.connect(base)
    sql = 'SELECT DATE,OPEN,HIGH,LOW,CLOSE,VOLUME from price_history where (SYMBOL="'+symbol+'") and (strftime("%Y", DATE) = "'+str(year)+'") order by DATE'
    df = pd.read_sql(sql,conn)
    if (len(df) == 0):
        return None
    first_price = df['OPEN'][0]
    last_price = df['CLOSE'][len(df)-1]
    max_price = df['HIGH'].max()
    max_volume = df['VOLUME'].max()
    mean_price = df['CLOSE'].mean()
    mean_volume = df['VOLUME'].mean()
    median_price = df['CLOSE'].median()
    median_volume = df['VOLUME'].median()
    min_price = df['LOW'].min()
    min_volume = df['VOLUME'].min()
    std_price = df['OPEN'].std()
    std_volume = df['VOLUME'].std()
    y = 0
    return first_price,last_price,max_price,max_volume,mean_price,mean_volume,median_price,median_volume,min_price,min_volume,std_price,std_volume,y

In [6]:
def get_sector_industry(base,symbol):
    conn = sqlite3.connect(base)
    sql='select SECTOR,INDUSTRY from sectors where SYMBOL="'+symbol+'"'
    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchone()
    conn.close()
    if results:
        sector = results[0]
        industry = results[1]
        return sector,industry
    else:
        return None

In [7]:
def get_div(base,symbol,year):
    dividends=0
    conn = sqlite3.connect(base)
    sql='select sum(DIVIDENDS) from dividends_history where SYMBOl="'+symbol+'" and (strftime("%Y", DATE) = "'+str(year)+'")'
    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchone()
    conn.close()
    if results:
        dividends=results[0]
    if dividends:
        return dividends
    else:
        return 0

In [8]:
def get_frs(base,year):
    conn = sqlite3.connect(base)
    sql='select RATE from FRS where (strftime("%Y", DATE) = "'+str(year)+'")'
    frs_df = pd.read_sql(sql,conn)
    frs_min = frs_df['RATE'].min()
    frs_max = frs_df['RATE'].max()
    frs_mean = frs_df['RATE'].mean()
    return frs_min,frs_max,frs_mean

In [9]:
def get_oil_gold(base,year):
    conn = sqlite3.connect(base)
    sql='select OPEN,HIGH,LOW,CLOSE from comodity_history where (SYMBOL="CL=F") and (strftime("%Y", DATE) = "'+str(year)+'")'
    df =  pd.read_sql(sql,conn)
    oil_min = df['LOW'].min()
    oil_max = df['HIGH'].max()
    oil_mean = df['OPEN'].mean()
    sql='select OPEN,HIGH,LOW,CLOSE from comodity_history where (SYMBOL="GC=F") and (strftime("%Y", DATE) = "'+str(year)+'")'
    df =  pd.read_sql(sql,conn)
    gold_min = df['LOW'].min()
    gold_max = df['HIGH'].max()
    gold_mean = df['OPEN'].mean()
    return oil_min,oil_max,oil_mean,gold_min,gold_max,gold_mean

# Основной цикл

In [10]:
%%time
row_list = []
for symbol in SP_SYMBOLS:
    sec_in = get_sector_industry(base,symbol)
    if sec_in:
        sector,industry = sec_in
    else:
        sector = 'no'
        industry = 'no'
   
    frs_min,frs_max,frs_mean = get_frs(base,year)
    values = get_price_history(base,symbol,year)
    prev_values = get_price_history(base,symbol,year-1)
    if (values is not None) and (prev_values is not None):
        first_price,last_price,max_price,max_volume,mean_price,mean_volume,median_price,median_volume,min_price,min_volume,std_price,std_volume,y = values
        prev_first_price,prev_last_price,prev_max_price,prev_max_volume,prev_mean_price,prev_mean_volume,prev_median_price,prev_median_volume,prev_min_price,prev_min_volume,prev_std_price,prev_std_volume,prev_y = prev_values
        div = get_div(base,symbol,year)
        prev_div = get_div(base,symbol,year-1)
        dict1 = {'symbol_year':symbol+'_'+str(year),
                 'sector':sector,
                 'industry':industry,
                 'last_price':last_price,
                 'max_price':max_price,
                 'max_volume':max_volume,
                 'mean_price':mean_price,
                 'mean_volume':mean_volume,
                 'median_price':median_price,
                 'median_volume':median_volume,
                 'min_price':min_price,
                 'min_volume':min_volume,
                 'std_price':std_price,
                 'std_volume':std_volume,
                 'dividends':div,
                 'prev_dividends':prev_div,
                 'prev_last_price':prev_last_price,
                 'prev_max_price':prev_max_price,
                 'prev_max_volume':prev_max_volume,
                 'prev_mean_price':prev_mean_price,
                 'prev_mean_volume':prev_mean_volume,
                 'prev_median_price':prev_median_price,
                 'prev_median_volume':prev_median_volume,
                 'prev_min_price':prev_min_price,
                 'prev_min_volume':prev_min_volume,
                 'prev_std_price':prev_std_price,
                 'prev_std_volume':prev_std_volume,
                 'frs_min':frs_min,
                 'frs_max':frs_max,
                 'frs_mean':frs_mean,
                 'year':year,
                 'y':y}
        row_list.append(dict1)
DATA = pd.DataFrame(row_list)

CPU times: user 7.81 s, sys: 1.2 s, total: 9.01 s
Wall time: 9.01 s


In [11]:
len(DATA)

458

In [12]:
DATA = DATA.dropna()

In [13]:
DATA = DATA.query('min_price > 1')

In [14]:
DATA['frs_int'] = round(DATA['frs_mean'])

In [15]:
DATA['frs_int']= DATA['frs_int'].astype(int)

In [16]:
len(DATA)

457

Цикл добавления комодитиз

In [17]:
first_year=year
last_year=year
row_list = []
conn = sqlite3.connect(base)
sql='select distinct symbol from comodity_history'
cursor = conn.cursor()
cursor.execute(sql)
comoditilist = cursor.fetchall()
for year in range(first_year,last_year+1):
    dict={}
    dict['year']=year
    for comodity in comoditilist:
        sql='select OPEN,HIGH,LOW,CLOSE from comodity_history where (strftime("%Y", DATE) = "'+str(year)+'") and (SYMBOL="'+comodity[0]+'")'
        df=pd.read_sql(sql,conn)
        dict[comodity[0]+'_mean']=df['OPEN'].mean()
        dict[comodity[0]+'_max']=df['HIGH'].max()
        dict[comodity[0]+'_min']=df['LOW'].min()
    row_list.append(dict)
DATA_COM = pd.DataFrame(row_list)
conn.close()    

In [18]:
DATA = DATA.merge(DATA_COM)

In [19]:
DATA=DATA.drop('year',axis=1)

In [20]:
len(DATA)

457

подключаем финансовые показатели

In [21]:
conn = sqlite3.connect(base)
DATA_F = pd.read_sql('SELECT date(DATE,"-7 days") as DATE,SYMBOL,KEY,VALUE from FINANCE',conn)

In [22]:
DATA_F['year']=DATA_F['DATE'].str.split('-').str[0]

In [23]:
DATA_F_PREV = DATA_F.copy(deep=True)

In [24]:
DATA_F_PREV['year'] = DATA_F_PREV['year'].astype(int)+1

In [25]:
DATA_F_PREV['year'] = DATA_F_PREV['year'].astype(str)

In [26]:
DATA_F_PREV['KEY'] = 'prev_'+DATA_F_PREV['KEY']

In [27]:
DATA_F = pd.concat([DATA_F_PREV,DATA_F], axis=0) 

In [28]:
DATA_F['symbol_year']=DATA_F['SYMBOL']+'_'+DATA_F['year']

In [29]:
DATA_F = DATA_F.drop('DATE',axis=1)
DATA_F = DATA_F.drop('SYMBOL',axis=1)

In [30]:
DATA_F = DATA_F.drop('year',axis=1)

In [31]:
PIVOT = pd.pivot_table(DATA_F,index='symbol_year',columns='KEY',aggfunc='mean')

In [32]:
PIVOT = PIVOT.reset_index().T.reset_index().drop('level_0',axis=1).T

In [33]:
PIVOT = PIVOT.T.set_index(PIVOT.T.columns[0]).T

In [34]:
PIVOT = PIVOT.rename(columns={PIVOT.columns[0]: 'symbol_year'})

In [35]:
DATA=DATA.merge(PIVOT)

In [36]:
len(DATA)

456

In [37]:
DATA = DATA.dropna()

In [38]:
len(DATA)

432

In [39]:
DATA.to_csv('y_test.csv',index=None)

In [40]:
DATA

,symbol_year,sector,industry,last_price,max_price,max_volume,mean_price,mean_volume,median_price,median_volume,...,totalDebt,totalInvestments,totalLiabilities,totalLiabilitiesAndStockholdersEquity,totalNonCurrentAssets,totalNonCurrentLiabilities,totalOtherIncomeExpensesNet,totalStockholdersEquity,weightedAverageShsOut,weightedAverageShsOutDil
0,AAPL_2020,Technology,Consumer Electronics,132.690002,138.789993,426884800.0,95.347075,1.575816e+08,91.632500,138023400.0,...,1.12436e+11,1.05854e+11,2.58549e+11,3.23888e+11,1.80175e+11,1.53157e+11,-8.7e+07,6.5339e+10,1.73521e+10,1.75282e+10
1,MSFT_2020,Technology,Software—Infrastructure,222.419998,232.860001,97073600.0,193.026126,3.766292e+07,201.910004,32790800.0,...,6.3327e+10,1.25916e+11,1.83007e+11,3.01311e+11,1.19396e+11,1.10697e+11,2.668e+09,1.18304e+11,7.61e+09,7.683e+09
2,AMZN_2020,Consumer Cyclical,Internet Retail,3256.929932,3552.250000,15567300.0,2680.855096,4.931702e+06,2890.300049,4526600.0,...,3.1875e+10,8.4548e+10,2.27791e+11,3.21195e+11,1.88462e+11,1.01406e+11,2.371e+09,9.3404e+10,5e+08,5.1e+08
3,FB_2020,Communication Services,Internet Content & Information,273.160004,304.670013,76343900.0,234.550870,2.244645e+07,235.940002,20142500.0,...,1.0654e+10,8.8756e+10,3.1026e+10,1.59316e+11,8.3646e+10,9.631e+09,5.09e+08,1.2829e+11,2.85e+09,2.888e+09
4,JPM_2020,Financial Services,Banks—Diversified,127.070000,141.100006,49725200.0,106.515534,1.923107e+07,100.059998,16358500.0,...,3.26893e+11,1.13881e+12,3.10672e+12,3.38607e+12,8.0537e+10,0,0,2.79354e+11,3.2215e+09,3.2215e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,UAA_2020,Consumer Cyclical,Apparel Manufacturing,17.170000,21.959999,44669800.0,12.830909,7.445738e+06,11.160000,6573200.0,...,1.84297e+09,0,3.35464e+09,5.03063e+09,1.17419e+09,1.10194e+09,1.68153e+08,1.67599e+09,2.22927e+08,2.22927e+08
452,UAA_2020,Consumer Cyclical,Apparel Manufacturing,17.170000,21.959999,44669800.0,12.830909,7.445738e+06,11.160000,6573200.0,...,1.84297e+09,0,3.35464e+09,5.03063e+09,1.17419e+09,1.10194e+09,1.68153e+08,1.67599e+09,2.22927e+08,2.22927e+08
453,SIG_2020,Consumer Cyclical,Luxury Goods,27.270000,32.220001,25334100.0,17.624704,2.376670e+06,17.860001,1844800.0,...,6.115e+08,2.04e+07,4.4595e+09,6.2991e+09,3.1443e+09,2.8069e+09,7e+06,1.8396e+09,5.2e+07,5.2e+07
454,PDCO_2020,Healthcare,Medical Distribution,29.629999,33.470001,4727200.0,23.258933,1.154981e+06,23.570000,985600.0,...,6.3762e+08,2.14915e+08,1.88123e+09,2.71535e+09,1.17258e+09,8.06335e+08,4.2211e+07,8.34117e+08,9.4154e+07,9.4154e+07
